In [2]:
import duckdb
import json
import requests
import geopandas as gpd
import pandas as pd
import os
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# Dictionary of all country codes and their corresponding country names
countries_dict = {
  "VA": "Vatican_City",
  "BV": "Bouvet_Island",
  "GS": "South_Georgia_And_South_Sandwich_Islands",
  "TK": "Tokelau",
  "PN": "Pitcairn_Islands",
  "WF": "Wallis_And_Futuna",
  "TV": "Tuvalu",
  "AX": "Aland_Islands",
  "KI": "Kiribati",
  "NU": "Niue",
  "BL": "Saint_Barthelemy",
  "AQ": "Antarctica",
  "MF": "Saint_Martin",
  "IO": "British_Indian_Ocean_Territory",
  "SX": "Sint_Maarten",
  "SM": "San_Marino",
  "JE": "Jersey",
  "NF": "Norfolk_Island",
  "MO": "Macau",
  "CX": "Christmas_Island",
  "FO": "Faroe_Islands",
  "MS": "Montserrat",
  "GL": "Greenland",
  "GG": "Guernsey",
  "AD": "Andorra",
  "TC": "Turks_And_Caicos_Islands",
  "BQ": "Bonaire",
  "UM": "US_Minor_Outlying_Islands",
  "PM": "Saint_Pierre_And_Miquelon",
  "AS": "American_Samoa",
  "CC": "Cocos_Islands",
  "SJ": "Svalbard_And_Jan_Mayen",
  "CP": "Clipperton_Island",
  "FK": "Falkland_Islands",
  "HT": "Haiti",
  "YT": "Mayotte",
  "VG": "British_Virgin_Islands",
  "LC": "Saint_Lucia",
  "AG": "Antigua_And_Barbuda",
  "GD": "Grenada",
  "KN": "Saint_Kitts_And_Nevis",
  "VC": "Saint_Vincent_And_The_Grenadines",
  "DM": "Dominica",
  "SB": "Solomon_Islands",
  "TO": "Tonga",
  "WS": "Samoa",
  "VU": "Vanuatu",
  "FJ": "Fiji",
  "CK": "Cook_Islands",
  "SH": "Saint_Helena",
  "PW": "Palau",
  "NR": "Nauru",
  "LI": "Liechtenstein",
  "MH": "Marshall_Islands",
  "CU": "Cuba",
  "IM": "Isle_Of_Man",
  "BM": "Bermuda",
  "SC": "Seychelles",
  "GM": "Gambia",
  "GQ": "Equatorial_Guinea",
  "GW": "Guinea_Bissau",
  "BF": "Burkina_Faso",
  "BI": "Burundi",
  "UG": "Uganda",
  "RW": "Rwanda",
  "MW": "Malawi",
  "LS": "Lesotho",
  "ZM": "Zambia",
  "AL": "Albania",
  "CF": "Central_African_Republic",
  "ML": "Mali",
  "TD": "Chad",
  "GN": "Guinea",
  "LR": "Liberia",
  "SL": "Sierra_Leone",
  "KM": "Comoros",
  "DJ": "Djibouti",
  "CV": "CapeVerde",
  "SN": "Senegal",
  "TG": "Togo",
  "BJ": "Benin",
  "CI": "Ivory_Coast",
  "NE": "Niger",
  "SO": "Somalia",
  "ET": "Ethiopia",
  "SZ": "Eswatini",
  "ZW": "Zimbabwe",
  "NA": "Namibia",
  "MG": "Madagascar",
  "MZ": "Mozambique",
  "SS": "South_Sudan",
  "BD": "Bangladesh",
  "NP": "Nepal",
  "BT": "Bhutan",
  "AF": "Afghanistan",
  "KH": "Cambodia",
  "KG": "Kyrgyzstan",
  "AM": "Armenia",
  "GE": "Georgia",
  "AZ": "Azerbaijan",
  "BY": "Belarus",
  "LV": "Latvia",
  "EE": "Estonia",
  "SI": "Slovenia",
  "SK": "Slovakia",
  "LT": "Lithuania",
  "BG": "Bulgaria",
  "MK": "North_Macedonia",
  "BA": "Bosnia_And_Herzegovina",
  "ME": "Montenegro",
  "XK": "Kosovo",
  "RS": "Serbia",
  "HR": "Croatia",
  "MD": "Moldova",
  "RO": "Romania",
  "GR": "Greece",
  "CY": "Cyprus",
  "MT": "Malta",
  "IE": "Ireland",
  "LU": "Luxembourg",
  "CH": "Switzerland",
  "NZ": "New_Zealand",
  "UY": "Uruguay",
  "PY": "Paraguay",
  "BO": "Bolivia",
  "CR": "Costa_Rica",
  "SV": "El_Salvador",
  "NI": "Nicaragua",
  "GT": "Guatemala",
  "HN": "Honduras",
  "PE": "Peru",
  "EC": "Ecuador",
  "CL": "Chile",
  "AR": "Argentina",
  "JM": "Jamaica",
  "DO": "Dominican_Republic",
  "AW": "Aruba",
  "CW": "Curacao",
  "BS": "Bahamas",
  "TT": "Trinidad_And_Tobago",
  "PR": "Puerto_Rico",
  "KY": "Cayman_Islands",
  "BB": "Barbados",
  "GF": "French_Guiana",
  "GP": "Guadeloupe",
  "MQ": "Martinique",
  "VI": "US_Virgin_Islands",
  "GU": "Guam",
  "MP": "Northern_Mariana_Islands",
  "PF": "French_Polynesia",
  "RE": "Reunion",
  "GI": "Gibraltar",
  "MA": "Morocco",
  "TN": "Tunisia",
  "DZ": "Algeria",
  "LY": "Libya",
  "SD": "Sudan",
  "IR": "Iran",
  "SY": "Syria",
  "YE": "Yemen",
  "IQ": "Iraq",
  "LB": "Lebanon",
  "JO": "Jordan",
  "AE": "United_Arab_Emirates",
  "KW": "Kuwait",
  "QA": "Qatar",
  "BH": "Bahrain",
  "OM": "Oman",
  "SA": "Saudi_Arabia",
  "TL": "TimorLeste",
  "BN": "Brunei",
  "TM": "Turkmenistan",
  "AU": "Australia",
  "IS": "Iceland",
  "PT": "Portugal",
  "ES": "Spain",
  "BE": "Belgium",
  "NO": "Norway",
  "FI": "Finland",
  "SE": "Sweden",
  "NL": "Netherlands",
  "DK": "Denmark",
  "CZ": "Czech_Republic",
  "HU": "Hungary",
  "PL": "Poland",
  "AT": "Austria",
  "DE": "Germany",
  "UA": "Ukraine",
  "BY": "Belarus",
  "TH": "Thailand",
  "AO": "Angola",
  "GH": "Ghana",
  "TZ": "Tanzania",
  "VN": "Vietnam",
  "MY": "Malaysia",
  "KH": "Cambodia",
  "ID": "Indonesia",
  "IL": "Israel",
  "HK": "Hong_Kong",
  "SG": "Singapore",
  "TW": "Taiwan",
  "KR": "South_Korea",
  "JP": "Japan",
  "PH": "Philippines",
  "US": "United_States",
  "CN": "China",
  "IN": "India"
}
# for key in countries_dict:
#     print(key)
#     print(countries_dict[key])

In [ ]:

country_tile_dict = {} # Dictionary to get the number of countries and the tiles that are not empty
for target_country in countries_dict: # Loop through each country
    
    ONEGEO_API_KEY = "API KEY HERE"  # Replace with your actual One Geo API key
    country_code = target_country # "NG" Example
    country_name = str(countries_dict[target_country]) # "Nigeria" Example
    min_height = 30  # meters
    tile_size = 0.5  # degrees

    # Connect to DUCKDB and load country boundaries from overture
    con = duckdb.connect()
    con.install_extension("spatial")
    con.load_extension("spatial")
    con.sql("LOAD spatial;")
    con.sql("SET s3_region='us-west-2';")

    # Load country boundary and bounding box
    con.sql(f"""
    CREATE OR REPLACE TABLE bounds AS (
        SELECT
            id AS division_id, names.primary, geometry, bbox
        FROM
            read_parquet('s3://overturemaps-us-west-2/release/2025-06-25.0/theme=divisions/type=division_area/*.parquet')
        WHERE
            subtype = 'country'
            AND country = '{country_code}'
    );
    """)

    # Set bounding box variables
    con.sql("SET variable xmin = (SELECT bbox.xmin FROM bounds);") # Min Longitude
    con.sql("SET variable ymin = (SELECT bbox.ymin FROM bounds);") # Min Latitude
    con.sql("SET variable xmax = (SELECT bbox.xmax FROM bounds);") # Max Longitude
    con.sql("SET variable ymax = (SELECT bbox.ymax FROM bounds);") # Max Latitude
    con.sql("SET variable boundary = (SELECT geometry FROM bounds);")

    # Retrieve bounding box values
    bbox_values = con.sql("""
        SELECT 
            getvariable('xmin') AS xmin,
            getvariable('ymin') AS ymin,
            getvariable('xmax') AS xmax,
            getvariable('ymax') AS ymax
    """).fetchall()

    xmin, ymin, xmax, ymax = bbox_values[0]
    print(f"Bounding box: xmin={xmin}, ymin={ymin}, xmax={xmax}, ymax={ymax}")

    # Tile the bounding box 
    tiles = []
    lat = ymin
    while lat < ymax:
        lon = xmin
        while lon < xmax:
            tile = (lon, lat, min(lon + tile_size, xmax), min(lat + tile_size, ymax))
            tiles.append(tile)
            lon += tile_size
        lat += tile_size

    print(f"Generated {len(tiles)} tiles.")

     

    # Filter tiles that intersect with the country's geometry 
    valid_tiles = []
    for tile in tiles:
        west, south, east, north = tile
        # MakeEnvelope creates a rectangle for the tile
        intersects = con.sql(f"""
            SELECT ST_Intersects(
                ST_MakeEnvelope({west}, {south}, {east}, {north}), 
                getvariable('boundary')
            ) AS intersects
        """).fetchone()[0]

        if intersects:
            valid_tiles.append(tile)

    print(f"{len(valid_tiles)} tiles intersect with {country_name}'s boundary.")

    country_tile_dict[country_name] = [] # each country is assigned an empty list as its value to collect all the tiles with features
    # Call one geo api on each valid tile
    for i, tile in enumerate(valid_tiles):
        west, south, east, north = tile
        bbox_str = f"{west},{south},{east},{north}"
        url = "https://data.onegeo.co/api/"
        params = {
            "token": ONEGEO_API_KEY,
            "bbox": bbox_str,
            "min_height": min_height,
            "types": "commerce,commerce:office,commerce:retail,residence,other,industry:storage",
            "show_sources": "true"
        }

        try:
            response = requests.get(url, params=params)
            if response.status_code == 200:
                data = response.json()
                features = data.get("features", [])
                # create geojson file for tile with features
                if features:
                    country_tile_dict[country_name].append(i+1) # add the tile number that has features to the dictionary to be used for merging files

                    output_file = f"{country_name}_tile_{i+1}.geojson"
                    with open(output_file, "w", encoding="utf-8") as f:
                        json.dump(data, f, indent=2)
                    print(f"Saved: {output_file}")
                else:
                    print(f"No features returned for chunk {i+1}, skipping file.")

        except Exception as ex:
                print(f"Exception for chunk {i+1}: {ex}")

In [ ]:
# Append the geojson files for each country into one single file

for country_name, tiles_list in country_tile_dict.items():
    first_geojson = gpd.read_file(fr"C:\Users\karim\OneDrive\Documents\Propeterra_Data\OneGeo_Buildings\{country_name}_tile_{tiles_list[0]}.geojson")
    print(f"Working on {country_name}")
    # Goes through the rest of the tiles and merges them with each other to get one single geojson file for each country
    for tile in range(1,len(tiles_list)):
        next_geojson = gpd.read_file(fr"C:\Users\karim\OneDrive\Documents\Propeterra_Data\OneGeo_Buildings\{country_name}_tile_{tiles_list[tile]}.geojson")
        result = pd.concat([first_geojson, next_geojson])
        first_geojson = result
    
    output_dir = r'C:\Users\karim\OneDrive\Documents\Propeterra_Data\OneGeo_Buildings' 
    output_file = os.path.join(output_dir, f'{country_name}_buildings.geojson')
    gdf = first_geojson.to_crs("EPSG:4087") # to set up the projection
    gdf.to_file(output_file, driver='GeoJSON')


In [3]:

def get_city(row):
    location = row["location"]
    try:
        address = location.raw.get("address", {})
        return (
            address.get("city") or
            address.get("town") or
            address.get("village") or
            address.get("municipality") or
            address.get("county") or
            " "
        )
    except Exception:
        return " "
    
# Reverse Geocode 

gdf = gpd.read_file(fr"C:\Users\karim\OneDrive\Documents\Propeterra_Data\OneGeo_Buildings\Dubai_One_Geo_Test_Data.geojson")

# Convert to EPSG:4326 for reverse geocoding to get accurate addresses
gdf = gdf.to_crs("EPSG:4326")

gdf["lon"] = gdf.geometry.representative_point().x # used representative point to make sure that centroid is inside the building
gdf["lat"] = gdf.geometry.representative_point().y

# Reverse geocoding (user_agent could be anyname)
geolocator = Nominatim(user_agent="propeterra_geocoder", timeout=10)
reverse_geocode = RateLimiter(geolocator.reverse, min_delay_seconds=0.1)

# Generate location column 
gdf["location"] = gdf.apply(lambda row: reverse_geocode(f"{row['lat']}, {row['lon']}"), axis=1)

gdf["city"] = gdf.apply(get_city, axis = 1)

# Convert to EPSG:4087 for Visualization in Cesium
gdf = gdf.to_crs("EPSG:4087")

# Save to GeoJSON
output_path = fr"C:\Users\karim\OneDrive\Documents\Propeterra_Data\OneGeo_Buildings\Dubai_buildings_with_addresses.geojson"
gdf.to_file(output_path, driver="GeoJSON")
